# Post-Ictal Heart Rate Oscillations in Partial Epilepsy

In [16]:
import os
import wfdb
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
from hrvanalysis import get_time_domain_features,get_frequency_domain_features, plot_psd

import warnings
warnings.filterwarnings("ignore")

In [17]:
if False:
    # baixando os arquivos de HRO (heart rate oscillations).
    wfdb.dl_database("szdb", os.path.join(os.getcwd(), "szdb"))

# Calculo dos tempos onde há convulsão

In [18]:
a = ['sz01 00:14:36 00:16:12', # 1
    'sz02 01:02:43 01:03:43', # 2
    'sz02 02:55:51 02:56:16', # 2
    'sz03 01:24:34 01:26:22', # 3
    'sz03 02:34:27 02:36:17', # 3
    'sz04 00:20:10 00:21:55', # 4
    'sz05 00:24:07 00:25:30', # 5
    'sz06 00:51:25 00:52:19', # 6
    'sz06 02:04:45 02:06:10', # 6
    'sz07 01:08:02 01:09:31'] # 7

def _(string):

    start = string.split()[1].split(':')
    end = string.split()[2].split(':')
    
    start_s = int(start[0])*60*60 + int(start[1])*60 + int(start[2])
    end_s = int(end[0])*60*60 + int(end[1])*60 + int(end[2])
    
    #print(start_s, end_s)
    return [start_s, end_s]
    
tempos = []
for string in a:
    tempos.append(_(string))
                  
tempos = [[[876, 972]],  #1
 [[3763, 3823],[10551, 10576]], # 2
 [[5074, 5182],[9267, 9377]], # 3
 [[1210, 1315]], # 4
 [[1447, 1530]], # 5
 [[3085, 3139],[7485, 7570]], # 6
 [[4082, 4171]]] # 7

tempos

[[[876, 972]],
 [[3763, 3823], [10551, 10576]],
 [[5074, 5182], [9267, 9377]],
 [[1210, 1315]],
 [[1447, 1530]],
 [[3085, 3139], [7485, 7570]],
 [[4082, 4171]]]

# Obtenção da VFC (variabilidade da frequência cardíaca) a partir do arquivo de anotações

In [19]:
# 140 media de amostras 'C' (1).
def _process_vfc(pandas, tempos):
    
    """
    
    """
    
    indexes_list = []
    #vfc[(vfc.tempo >= tempo[0]) & (vfc.tempo <= tempo[1])]
    # pegando indexes onde há convulsão.
    for tempo in tempos:
        indexes = pandas.index[(pandas['tempo'] >= tempo[0]) & (pandas['tempo'] <= tempo[1])].tolist()
        print('indexes "C" (1): ',np.min(indexes), np.max(indexes))
        print('dif:',np.max(indexes) - np.min(indexes))
        indexes_list.extend(indexes)
        
    #print(indexes_list)
   
    print('tamanho amostras',len(pandas))
    print('--------------------------')
    # criando uma coluna auxiliar para usar apply.
    pandas['index1'] = pandas.index
    # criando a coluna labels, recebe 'C' se o index da linha esta na lista, se não 'N'.
    #pandas['labels'] = pandas.apply(lambda row: 'C' if pandas.index(row) in indexes else 'N', axis=1)
    pandas['labels'] = pandas.apply(lambda row: 1 if row.index1 in indexes_list else 0, axis=1)
    pandas = pandas.drop(columns = ['index1'])
    # retorna o dataset anotado.
    return pandas


In [20]:
def most_frequent(List):
    """
    obtem a label com maior numero de ocorrencias em uma lista ('N' (0), 'C' (1)).
    """
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

In [21]:
df_vfc = pd.DataFrame()
for i in range(1,8):
    
    """
    para cada dataset, carrega as anotações, converte para pandas dataframe e cria a coluna das labels ('N' (0): normal, 'C' (1): convulsão).
    
    retorna um dataframe com todos os datasets juntos anotados (df_vfc).
    """
    
    path = "szdb/sz0"+str(i)
    
    record = wfdb.rdrecord(path) # Efetuamos a leitura do arquivo apenas para obter o número total de pontos e fs.
    fs=record.fs
    nr_sampl=record.sig_len
    del record # Liberamos memória
    print('Número de amostras=',nr_sampl,', taxa de amostragem=',fs)
    
    # Obtem as anotações do arquivo sz01.ari
    sampfrom=0
    sampto=nr_sampl  # pega todo o sinal.
    annotation = wfdb.rdann(path, 'ari', sampfrom=sampfrom, sampto=sampto) 
    annotation.fs = fs # Ajustamos o tempo das anotações para a mesma taxa de amostragem do sinal
    
    # Efetuamos novamente a leitura de todo o sinal
    annotation = wfdb.rdann(path, 'ari')
    r_times=annotation.sample #criamos um array com os tempos (em número de amostras)
    vfc = pd.DataFrame(r_times,columns = ['sample'])
    vfc['tempo']= vfc['sample']/fs # convertemos o número da amostra em segundos
    vfc['vfc'] =  vfc['tempo'].shift(-1)-vfc['tempo']
    vfc = vfc.fillna(0) # remove NaN substituindo por zeros.
                            
    tempo_t = tempos[i-1]                       
    vfc = _process_vfc(pandas = vfc, tempos = tempo_t)
    df_vfc = pd.concat([df_vfc, vfc])

Número de amostras= 1079998 , taxa de amostragem= 200
indexes "C" (1):  1088 1273
dif: 185
tamanho amostras 8385
--------------------------
Número de amostras= 2519998 , taxa de amostragem= 200
indexes "C" (1):  3875 3959
dif: 84
indexes "C" (1):  10842 10877
dif: 35
tamanho amostras 13195
--------------------------
Número de amostras= 2711998 , taxa de amostragem= 200
indexes "C" (1):  5790 5957
dif: 167
indexes "C" (1):  10954 11172
dif: 218
tamanho amostras 16384
--------------------------
Número de amostras= 1079998 , taxa de amostragem= 200
indexes "C" (1):  1226 1408
dif: 182
tamanho amostras 6229
--------------------------
Número de amostras= 1080006 , taxa de amostragem= 200
indexes "C" (1):  2164 2324
dif: 160
tamanho amostras 8076
--------------------------
Número de amostras= 2159998 , taxa de amostragem= 200
indexes "C" (1):  3729 3796
dif: 67
indexes "C" (1):  8894 8991
dif: 97
tamanho amostras 12758
--------------------------
Número de amostras= 1439998 , taxa de amostrag

In [22]:
# Extrai atributos no domínio tempo

"""
Em uma janela de 140 amostras (media de amostras onde há convulsão),
obtem as features no tempo e na frequencia e cria um pandas dataframe.
"""

df_final = pd.DataFrame()
#for i in range(0:len(df_vfc):140):
for index ,i in enumerate(range(0,len(df_vfc),140)):    
    
    #df_aux = pd.DataFrame()
    label = most_frequent(List = df_vfc['labels'].to_list())

    RR_interval = df_vfc['vfc'][i:i+140] # pegando 140 em 140 amostras.
    td_features = get_time_domain_features(RR_interval);
    #display(td_features)
    # Extrai atributos no domínio da frequência
    fd_features = get_frequency_domain_features(RR_interval, sampling_frequency = 10);
    
    td_features.update(fd_features) 
        
    df_aux = pd.DataFrame(td_features,  index=[index])
    #df_aux['labels'].loc[index] = label
    df_aux['labels'] = label
    
    df_final = pd.concat([df_final,df_aux])
    
df_final.head()

,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,...,min_hr,std_hr,lf,hf,lf_hf_ratio,lfnu,hfnu,total_power,vlf,labels
0,0.833321,0.039281,0.035863,0,0.0,0,0.0,0.035865,0.8300,0.195,...,64864.864865,3394.000752,5.148006e-05,0.000118,0.435644,30.344828,69.655172,0.000186,1.638002e-05,0
1,0.823536,0.037897,0.027789,0,0.0,0,0.0,0.027790,0.8200,0.205,...,63829.787234,3326.000594,1.839424e-05,0.000042,0.435644,30.344828,69.655172,0.000066,5.852714e-06,0
2,0.848179,0.062893,0.040872,0,0.0,0,0.0,0.040872,0.8375,0.355,...,55045.871560,4832.735078,8.451331e-07,0.000002,0.435644,30.344828,69.655172,0.000003,2.689060e-07,0
3,0.810357,0.031232,0.025786,0,0.0,0,0.0,0.025786,0.8100,0.165,...,64864.864865,2782.041545,6.579590e-06,0.000015,0.435644,30.344828,69.655172,0.000024,2.093506e-06,0
4,0.801464,0.024381,0.022743,0,0.0,0,0.0,0.022743,0.8000,0.120,...,68965.517241,2244.251852,6.579590e-06,0.000015,0.435644,30.344828,69.655172,0.000024,2.093506e-06,0


In [25]:
df_final.to_csv('vfc_dataset.csv', index=False)